In [1]:
"""Temporary fix for importing modules from parent directory"""
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Example 00: Two-link

Integrating two-link, using variational integrator based on Discrete Euler-Lagrange


In [2]:
from varitop.problem import VaritopProblem
from varitop.misc import euler_quadrature
import casadi as cs
import numpy as np

In [3]:
nodes = 50
problem = VaritopProblem()
problem.set_nodes(nodes)
problem.set_quadrature(euler_quadrature)

In [4]:
qs = problem.create_state("q", 6)
vs = problem.create_velocity("v", 6)

In [5]:
q, v = qs[0], vs[0]
q, v

(SX([q_0, q_1, q_2, q_3, q_4, q_5]), SX([v_0, v_1, v_2, v_3, v_4, v_5]))

Here we are assuming all masses and lengths are equal to 1 for simplicity


In [6]:
kinetic_energy = cs.Function("E", [v], [0.5 * v.T @ v])
potential_energy = cs.Function("U", [q], [9.81 * (q[1] + q[3] + q[5])])
total_energy = cs.Function("H", [q, v], [kinetic_energy(v) + potential_energy(q)])
lagrangian = cs.Function("L", [q, v], [kinetic_energy(v) - potential_energy(q)])

In [7]:
phi0 = cs.Function("phi0", [q], [q[0]])
phi1 = cs.Function("phi1", [q], [(q[0] - q[2]) ** 2 + (q[1] - q[3]) ** 2 - 1])
phi2 = cs.Function("phi2", [q], [(q[2] - q[4]) ** 2 + (q[3] - q[5]) ** 2 - 1])

In [8]:
problem.add_constraint("=", phi0)
problem.add_constraint("=", phi1)
problem.add_constraint("=", phi2)

In [9]:
problem.set_continuous_lagrangian(lagrangian)
discrete_lagrangian = problem.get_discrete_lagrangian()
del_constraint = problem.get_del_residual()
del_constraint

Function(DEL:(q-1[6],q[6],q+1[6],dt)->(DEL_residual[6]) SXFunction)